Denoising Autoencode (DAE)

In [1]:
# 📌 Step 1: Install Required Libraries
!pip install tensorflow scikit-image opencv-python

In [2]:
# 📌 Step 2: Import Libraries
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from skimage.metrics import peak_signal_noise_ratio as psnr
from google.colab import drive

In [3]:
# 📌 Step 3: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 📌 Step 4: Define paths
input_base_dir = "/content/drive/MyDrive/Pneumonia Disease Detection Dataset"
output_base_dir = "/content/drive/MyDrive/Denoising Autoencode (DAE)"
noisy_dir = os.path.join(output_base_dir, "Noisy")
denoised_dir = os.path.join(output_base_dir, "Denoised")

In [5]:
# 📌 Step 5: Create folder structure for Noisy and Denoised images
for main_folder in [noisy_dir, denoised_dir]:
    for subset in ["Training Data", "Validation Data", "Testing Data"]:
        for cls in ["Normal", "Bacterial Pneumonia", "Viral Pneumonia"]:
            os.makedirs(os.path.join(main_folder, subset, cls), exist_ok=True)

In [6]:
# 📌 Step 6: Build the Denoising Autoencoder
def build_autoencoder():
    input_img = Input(shape=(128, 128, 1))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(inputs=input_img, outputs=decoded)
    model.compile(optimizer=Adam(), loss='binary_crossentropy')
    return model

autoencoder = build_autoencoder()
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 16)          │           4,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 128, 128, 1)         │             289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,193 (47.63 KB)

 Trainable params: 12,193 (47.63 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 📌 Step 7: Load training images and create noisy versions
def load_images(path):
    images = []
    for cls in os.listdir(path):
        cls_path = os.path.join(path, cls)
        for img_name in os.listdir(cls_path):
            try:
                img = load_img(os.path.join(cls_path, img_name), color_mode='grayscale', target_size=(128, 128))
                img_array = img_to_array(img) / 255.0
                images.append(img_array)
            except:
                pass
    return np.array(images)

x_train = load_images(os.path.join(input_base_dir, "Training Data"))
x_train_noisy = x_train + 0.2 * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)

In [8]:
# 📌 Step 8: Train Autoencoder
autoencoder.fit(x_train_noisy, x_train, epochs=10, batch_size=32, shuffle=True, verbose=1)

Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - loss: 0.6152
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5664
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5662
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5645
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5655
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5649
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.5639
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5632
Epoch 9/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5628
Epoch 10/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5643


In [9]:
# 📌 Step 9: Denoise and save images with PSNR calculation
def denoise_and_save_images(autoencoder, input_dir, noisy_dir, denoised_dir, subset_name):
    psnr_records = []

    for cls in os.listdir(input_dir):
        cls_input_path = os.path.join(input_dir, cls)
        cls_noisy_path = os.path.join(noisy_dir, subset_name, cls)
        cls_denoised_path = os.path.join(denoised_dir, subset_name, cls)

        for img_name in tqdm(os.listdir(cls_input_path), desc=f"{subset_name}/{cls}"):
            try:
                img_path = os.path.join(cls_input_path, img_name)
                img = load_img(img_path, color_mode='grayscale', target_size=(128, 128))
                img_array = img_to_array(img) / 255.0
                img_array = np.expand_dims(img_array, axis=0)

                noisy_img = img_array + 0.2 * np.random.normal(loc=0.0, scale=1.0, size=img_array.shape)
                noisy_img = np.clip(noisy_img, 0., 1.)

                denoised_img = autoencoder.predict(noisy_img)[0]

                # PSNR Calculation
                psnr_val = psnr(np.squeeze(img_array), np.squeeze(denoised_img))
                psnr_records.append({
                    "Subset": subset_name,
                    "Class": cls,
                    "Image": img_name,
                    "PSNR": psnr_val
                })

                # Save noisy image
                noisy_img_save = (noisy_img[0] * 255).astype(np.uint8)
                cv2.imwrite(os.path.join(cls_noisy_path, img_name), noisy_img_save)

                # Save denoised image
                denoised_img_save = (denoised_img * 255).astype(np.uint8)
                cv2.imwrite(os.path.join(cls_denoised_path, img_name), denoised_img_save)

            except Exception as e:
                print(f"Error processing {img_name}: {e}")
    return psnr_records

In [10]:
# 📌 Step 10: Apply to all datasets
all_psnr_records = []
for subset in ["Training Data", "Validation Data", "Testing Data"]:
    input_path = os.path.join(input_base_dir, subset)
    records = denoise_and_save_images(autoencoder, input_path, noisy_dir, denoised_dir, subset)
    all_psnr_records.extend(records)

Training Data/Bacterial Pneumonia:   0%|          | 0/1159 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step


Training Data/Bacterial Pneumonia:   0%|          | 1/1159 [00:00<13:53,  1.39it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


Training Data/Bacterial Pneumonia:   0%|          | 3/1159 [00:00<04:58,  3.87it/s]

...

Testing Data/Normal:  99%|█████████▉| 329/331 [01:13<00:00,  5.59it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


Testing Data/Normal: 100%|█████████▉| 330/331 [01:14<00:00,  5.42it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


Testing Data/Normal: 100%|██████████| 331/331 [01:14<00:00,  4.45it/s]


In [11]:
# 📌 Step 11: Save PSNR values to CSV
psnr_df = pd.DataFrame(all_psnr_records)

# Ensure output folder exists
csv_dir = os.path.join(output_base_dir)
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Denoising Autoencode (DAE) PSNR Results.csv")
psnr_df.to_csv(csv_path, index=False)

In [12]:
# 📌 Step 12: Summary
print(f"\n✅ PSNR results saved to: {csv_path}")
print("\n📊 Average PSNR by Subset & Class:")
print(psnr_df.groupby(["Subset", "Class"])["PSNR"].mean().round(2))


✅ PSNR results saved to: /content/drive/MyDrive/Denoising Autoencode (DAE)/Denoising Autoencode (DAE) PSNR Results.csv

📊 Average PSNR by Subset & Class:
Subset           Class              
Testing Data     Bacterial Pneumonia    25.67
                 Normal                 24.28
                 Viral Pneumonia        25.20
Training Data    Bacterial Pneumonia    25.66
                 Normal                 24.17
                 Viral Pneumonia        25.55
Validation Data  Bacterial Pneumonia    25.81
                 Normal                 24.03
                 Viral Pneumonia        25.03
Name: PSNR, dtype: float64


In [13]:
# 📌 Step 13: Print final folder structure preview
print("\n📁 Final folder structure:")
for root, dirs, files in os.walk(output_base_dir):
    level = root.replace(output_base_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}📁 {os.path.basename(root)}/")
    for f in files[:2]:
        print(f"{indent}    📄 {f}")
    if len(files) > 2:
        print(f"{indent}    ... ({len(files)} files)")


📁 Final folder structure:
📁 Denoising Autoencode (DAE)/
    📄 Denoising Autoencode (DAE) PSNR Results.csv
    📁 Noisy/
        📁 Training Data/
            📁 Normal/
                📄 Training_Data_Normal_0160.jpg
                📄 Training_Data_Normal_0161.jpg
                ... (1159 files)
            📁 Bacterial Pneumonia/
                📄 Training_Data_Bacterial_Pneumonia_0160.jpg
                📄 Training_Data_Bacterial_Pneumonia_0161.jpg
                ... (1159 files)
            📁 Viral Pneumonia/
                📄 Training_Data_Viral_Pneumonia_0160.jpg
                📄 Training_Data_Viral_Pneumonia_0161.jpg
                ... (1159 files)
        📁 Validation Data/
            📁 Normal/
                📄 Validation_Data_Normal_0001.jpg
                📄 Validation_Data_Normal_0002.jpg
                ... (166 files)
            📁 Bacterial Pneumonia/
                📄 Validation_Data_Bacterial_Pneumonia_0001.jpg
                📄 Validation_Data_Bacterial_Pneumonia_000